In [1]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
from ipyleaflet import *
import json
import requests 
from IPython.display import clear_output
from ipywidgets import *
import time
import numpy as np
import multiprocessing as mp

/anaconda2/lib/python2.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/anaconda2/lib/python2.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
## Make sure the jupyter kernel is running python2 
## Apparently (as of April. 2019) ipywidgets did not add widgetControl for python3, and the last cell will fail.
! python --version

Python 2.7.16 :: Anaconda, Inc.


In [3]:
onlyfiles = [f for f in listdir("spreadsheets/") if isfile(join("spreadsheets/", f))]
onlyfiles

['Drinking Water Watch Dataset.xlsx',
 'Public Potable Water Systems FINAL 06-22-2018_0.csv',
 'uw_self-cert_submittals.xlsx']

In [4]:
allfiles = {}
for a in onlyfiles:
    if ".xlsx" in a:
        allfiles[a.split(' ', 1)[0]] = pd.ExcelFile("spreadsheets/" + a)
    if ".csv" in a:
        print(a)
        allfiles[a.split(' ', 1)[0]] = pd.read_csv("spreadsheets/" + a, encoding='ISO-8859-1')

Public Potable Water Systems FINAL 06-22-2018_0.csv


In [5]:
alldata = {}
for key, b in allfiles.items():
    if type(b) is pd.ExcelFile:
        alldata[key] = {}
        for a in b.sheet_names:
            alldata[key][a] = b.parse(a)
    if type(b) is pd.DataFrame:
        alldata[key] = {}
        alldata[key][key] = b
print(alldata.keys())

['uw_self-cert_submittals.xlsx', 'Drinking', 'Public']


In [6]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

In [7]:
a = alldata['Drinking']["Table3_AllWellLocations"]
b = alldata["Public"]['Public']
print("Drinking data - well locations (found online) has shape: " + str(a.shape))
print("Public Potable water (Greg gave us) has shape " + str(b.shape))


Drinking data - well locations (found online) has shape: (29632, 10)
Public Potable water (Greg gave us) has shape (7898, 31)


In [8]:
c = a.merge(b, left_on='WATER_SYSTEM_NO', right_on='Water System No ')

In [9]:
c.keys()

Index([                                     u'REG_AGENCY',                                 u'WATER_SYSTEM_NO',                                        u'PWS_NAME',                          u'SYSTEM_ACTIVITY_STATUS',                                   u'FACILITY_NAME',                                    u'WSF_Activity',                                          u'WSF_ID',                                        u'WSF_Type',                                u'LATITUDE_MEASURE',                               u'LONGITUDE_MEASURE',                                u'Water System No ',                               u'Water System Name',                         u'Principal County Served',                 u'Federal Water System Type -CODE',                      u'Federal Water System Type ',                   u'State Water System Type -CODE',                         u'State Water System Type',                       u'Water System Status -CODE',                             u'Water System Status',
     

In [10]:
c.head(10)

,REG_AGENCY,WATER_SYSTEM_NO,PWS_NAME,SYSTEM_ACTIVITY_STATUS,FACILITY_NAME,WSF_Activity,WSF_ID,WSF_Type,LATITUDE_MEASURE,LONGITUDE_MEASURE,Water System No,Water System Name,Principal County Served,Federal Water System Type -CODE,Federal Water System Type,State Water System Type -CODE,State Water System Type,Water System Status -CODE,Water System Status,Owner Type -CODE,Owner Type,Primary Water Source Type -CODE,Primary Water Source Type,Residential Population,Non Transient Population,Transient Population,Total Population,Number of Service Connections Agricultural,Number of COMBINED Service Connections (CB),Number of Commercial (CM) Service Connections,Numer of Institutional Service Conections,Number of Residential Service Connections,Total Number of Service Connections,Fee Code,Fee Code Description,Date of Sanitary Survey visit (SNSV Visit Date),CITY,Treatment Plant Class-CODE,Treatment Plant Class,Distribution System Class-CODE,Distribution System Class
0,DISTRICT 01 - KLAMATH,CA0800526,REDWOOD PARK C.S.D.,A,WELL 01 XCLD,A,001,WL,41.575194,-124.066962,CA0800526,REDWOOD PARK C.S.D.,DEL NORTE,C,Community,C,Community,A,ACTIVE,L,Local,GW,Groundwater,200.0,NaN,NaN,200.0,NaN,44.0,NaN,NaN,NaN,44.0,SC,Small Community,5/7/2018,KLAMATH,NaN,NaN,D1,Distribution Operator Level 1
1,DISTRICT 01 - KLAMATH,CA0800526,REDWOOD PARK C.S.D.,A,WELL 02 XCLD,A,002,WL,41.575217,-124.066908,CA0800526,REDWOOD PARK C.S.D.,DEL NORTE,C,Community,C,Community,A,ACTIVE,L,Local,GW,Groundwater,200.0,NaN,NaN,200.0,NaN,44.0,NaN,NaN,NaN,44.0,SC,Small Community,5/7/2018,KLAMATH,NaN,NaN,D1,Distribution Operator Level 1
2,DISTRICT 01 - KLAMATH,CA0800526,REDWOOD PARK C.S.D.,A,WELL 03 - OUTSIDE - REPRESENTATIVE,A,003,WL,41.575224,-124.066872,CA0800526,REDWOOD PARK C.S.D.,DEL NORTE,C,Community,C,Community,A,ACTIVE,L,Local,GW,Groundwater,200.0,NaN,NaN,200.0,NaN,44.0,NaN,NaN,NaN,44.0,SC,Small Community,5/7/2018,KLAMATH,NaN,NaN,D1,Distribution Operator Level 1
3,DISTRICT 01 - KLAMATH,CA0800532,BIG ROCK C.S.D.,A,SMITH RIVER INFILTRATION GALLERY,A,001,WL,41.789707,-124.068578,CA0800532,BIG ROCK C.S.D.,DEL NORTE,C,Community,C,Community,A,ACTIVE,L,Local,GW,Groundwater,400.0,NaN,NaN,400.0,NaN,107.0,NaN,NaN,NaN,107.0,SC,Small Community,12/7/2016,CRESCENT CITY,T1,Treatment Operator Level 1,D1,Distribution Operator Level 1
4,DISTRICT 01 - KLAMATH,CA0800542,CRESCENT BEACH MOTEL,A,NEWEST WELL - REPRESENTATIVE,A,003,WL,41.741290,-124.165293,CA0800542,CRESCENT BEACH MOTEL,DEL NORTE,NC,Transient Non-Community,NC,Transient Non-Community,A,ACTIVE,P,Private,GW,Groundwater,NaN,NaN,70.0,70.0,NaN,2.0,NaN,NaN,NaN,2.0,N1,Transient-Noncommunity Water System,8/8/2017,CRESCENT CITY,T1,Treatment Operator Level 1,NR,Not Required
5,DISTRICT 01 - KLAMATH,CA0800542,CRESCENT BEACH MOTEL,A,WELL 02 - OLDEST XCLD,A,001,WL,41.741166,-124.165116,CA0800542,CRESCENT BEACH MOTEL,DEL NORTE,NC,Transient Non-Community,NC,Transient Non-Community,A,ACTIVE,P,Private,GW,Groundwater,NaN,NaN,70.0,70.0,NaN,2.0,NaN,NaN,NaN,2.0,N1,Transient-Noncommunity Water System,8/8/2017,CRESCENT CITY,T1,Treatment Operator Level 1,NR,Not Required
6,DISTRICT 01 - KLAMATH,CA0800542,CRESCENT BEACH MOTEL,A,INACTIVE WELL XCLD,I,004,WL,41.741376,-124.165494,CA0800542,CRESCENT BEACH MOTEL,DEL NORTE,NC,Transient Non-Community,NC,Transient Non-Community,A,ACTIVE,P,Private,GW,Groundwater,NaN,NaN,70.0,70.0,NaN,2.0,NaN,NaN,NaN,2.0,N1,Transient-Noncommunity Water System,8/8/2017,CRESCENT CITY,T1,Treatment Operator Level 1,NR,Not Required
7,DISTRICT 01 - KLAMATH,CA0800548,KLAMATH C.S.D.,A,HWY 101 WELL 01 - REPRESENTS 1 & 2,A,002,WL,41.527687,-124.039034,CA0800548,KLAMATH C.S.D.,DEL NORTE,C,Community,C,Community,A,ACTIVE,L,Local,GW,Groundwater,100.0,NaN,NaN,100.0,NaN,60.0,NaN,NaN,NaN,60.0,SC,Small Community,5/7/2018,CRESCENT CITY,NaN,NaN,D1,Distribution Operator Level 1
8,DISTRICT 01 - KLAMATH,CA0800548,KLAMATH C.S.D.,A,WELL 02 XCLD,A,001,WL,41.527765,-124.039111,CA0800548,KLAMATH C.S.D.,DEL NORTE,C,Community,C,Community,A,ACTIVE,L,Local,GW,Groundwat

In [11]:
### Filter out wells that have incomplete or zero latitude, longitude.

removed_null_na = c.dropna(subset=['LATITUDE_MEASURE', 'LONGITUDE_MEASURE'])
removed_null_na = removed_null_na[(removed_null_na['LATITUDE_MEASURE'] != 0) & (removed_null_na['LONGITUDE_MEASURE'] != 0)]
primary_merged = removed_null_na.copy(deep = True)
print( str(len(removed_null_na)) + " out of " + str(len(c)) + " rows have complete and nonzero latitude and longitude" )

19890 out of 23277 rows have complete and nonzero latitude and longitude


In [12]:
### Filter out wells that are abandoned or destroyed.
abandoned = primary_merged[primary_merged['FACILITY_NAME'].str.contains("ABANDONED|DESTROYED")]
print(str(len(abandoned)) + " out of " + str(len(primary_merged)) + " wells are abandoned or destroyed")
primary_merged = primary_merged[~(primary_merged['FACILITY_NAME'].str.contains("ABANDONED|DESTROYED"))]
print("updated # rows:" + str(len(primary_merged)))

3293 out of 19890 wells are abandoned or destroyed
updated # rows:16597


In [13]:
### fill all quantitative nan with zero.
fill_dict = {'Residential Population': 0, 'Non Transient Population': 0, 
                                       'Transient Population': 0, 'Total Population': 0, 'Number of Service Connections Agricultural': 0,
                                       'Number of COMBINED Service Connections (CB)': 0, 'Number of Commercial (CM) Service Connections': 0,
                                       'Numer of Institutional Service Conections': 0, 'Number of Residential Service Connections': 0,
                                       'Total Number of Service Connections': 0 }
primary_merged = primary_merged.fillna(value = fill_dict)
primary_merged.head(20)

,REG_AGENCY,WATER_SYSTEM_NO,PWS_NAME,SYSTEM_ACTIVITY_STATUS,FACILITY_NAME,WSF_Activity,WSF_ID,WSF_Type,LATITUDE_MEASURE,LONGITUDE_MEASURE,Water System No,Water System Name,Principal County Served,Federal Water System Type -CODE,Federal Water System Type,State Water System Type -CODE,State Water System Type,Water System Status -CODE,Water System Status,Owner Type -CODE,Owner Type,Primary Water Source Type -CODE,Primary Water Source Type,Residential Population,Non Transient Population,Transient Population,Total Population,Number of Service Connections Agricultural,Number of COMBINED Service Connections (CB),Number of Commercial (CM) Service Connections,Numer of Institutional Service Conections,Number of Residential Service Connections,Total Number of Service Connections,Fee Code,Fee Code Description,Date of Sanitary Survey visit (SNSV Visit Date),CITY,Treatment Plant Class-CODE,Treatment Plant Class,Distribution System Class-CODE,Distribution System Class
0,DISTRICT 01 - KLAMATH,CA0800526,REDWOOD PARK C.S.D.,A,WELL 01 XCLD,A,001,WL,41.575194,-124.066962,CA0800526,REDWOOD PARK C.S.D.,DEL NORTE,C,Community,C,Community,A,ACTIVE,L,Local,GW,Groundwater,200.0,0.0,0.0,200.0,0.0,44.0,0.0,0.0,0.0,44.0,SC,Small Community,5/7/2018,KLAMATH,NaN,NaN,D1,Distribution Operator Level 1
1,DISTRICT 01 - KLAMATH,CA0800526,REDWOOD PARK C.S.D.,A,WELL 02 XCLD,A,002,WL,41.575217,-124.066908,CA0800526,REDWOOD PARK C.S.D.,DEL NORTE,C,Community,C,Community,A,ACTIVE,L,Local,GW,Groundwater,200.0,0.0,0.0,200.0,0.0,44.0,0.0,0.0,0.0,44.0,SC,Small Community,5/7/2018,KLAMATH,NaN,NaN,D1,Distribution Operator Level 1
2,DISTRICT 01 - KLAMATH,CA0800526,REDWOOD PARK C.S.D.,A,WELL 03 - OUTSIDE - REPRESENTATIVE,A,003,WL,41.575224,-124.066872,CA0800526,REDWOOD PARK C.S.D.,DEL NORTE,C,Community,C,Community,A,ACTIVE,L,Local,GW,Groundwater,200.0,0.0,0.0,200.0,0.0,44.0,0.0,0.0,0.0,44.0,SC,Small Community,5/7/2018,KLAMATH,NaN,NaN,D1,Distribution Operator Level 1
3,DISTRICT 01 - KLAMATH,CA0800532,BIG ROCK C.S.D.,A,SMITH RIVER INFILTRATION GALLERY,A,001,WL,41.789707,-124.068578,CA0800532,BIG ROCK C.S.D.,DEL NORTE,C,Community,C,Community,A,ACTIVE,L,Local,GW,Groundwater,400.0,0.0,0.0,400.0,0.0,107.0,0.0,0.0,0.0,107.0,SC,Small Community,12/7/2016,CRESCENT CITY,T1,Treatment Operator Level 1,D1,Distribution Operator Level 1
4,DISTRICT 01 - KLAMATH,CA0800542,CRESCENT BEACH MOTEL,A,NEWEST WELL - REPRESENTATIVE,A,003,WL,41.741290,-124.165293,CA0800542,CRESCENT BEACH MOTEL,DEL NORTE,NC,Transient Non-Community,NC,Transient Non-Community,A,ACTIVE,P,Private,GW,Groundwater,0.0,0.0,70.0,70.0,0.0,2.0,0.0,0.0,0.0,2.0,N1,Transient-Noncommunity Water System,8/8/2017,CRESCENT CITY,T1,Treatment Operator Level 1,NR,Not Required
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,DISTRICT 01 - KLAMATH,CA0800605,WEST PARK PROPERTIES,A,WELL 02 - MAIN,A,001,WL,41.781063,-124.196232,CA0800605,WEST PARK PROPERTIES,DEL NORTE,C,Community,C,Community,A,ACTIVE,P,Private,GW,Groundwater,112.0,0.0,0.0,112.0,0.0,42.0,0.0,0.0,0.0,42.0,SC,Small Community,5/9/2018,CRESCENT CITY,NaN,NaN,D1,Distribution Operator Level 1
16,DISTRICT 01 - KLAMATH,CA0800608,TREES OF MYSTERY,A,WELL 01 - RAW,A,003,WL,41.583741,-124.086240,CA0800608,TREES OF MYSTERY,DEL NORTE,NC,Transient Non-Community,NC,Transient Non-Community,A,ACTIVE,P,Private,GW,Groundwater,0.0,0.0,200.0,200.0,0.0,7.0,0.0,0.0,0.0,7.0,N1,Transient-Noncommunity Water System,5/7/2018,KLAMATH,NaN,NaN,NR,Not Required
17,DISTRICT 01 - KLAMATH,CA0800615,KLAMATH CAMPER CORRAL,A,WELL 01 - PRIMARY,A,001,WL,41.521712,-124.036588,CA0800615,KLAMATH CAMPER CORRAL,DEL NORTE,NC,Transient Non-Community,NC,Transient Non-Community,A,ACTIVE,P,Private,GW,Groundwater,0.0,0.0,200.0,200.0,0.0,14.0,0.0,0.0,0.0,14.0,N1,Transient-Noncommunity Water System,8/7/2017,KLAMATH,NaN,NaN,NR,Not Required
18,DISTRICT 01 - KLAMATH,CA0800622,CHINOOK WATER SYSTEM,A,WELL 01,A,001,WL,41.54394

In [22]:
info_widgets = HTML(
    value=" Here's some useful info "
)

click_widget = Checkbox(
    description='Disable Updates',
    disabled=False
)

measure = MeasureControl(
    position='topleft',
    active_color = 'orange',
    primary_length_unit = 'kilometers'
)

def summarize(df, county_name):
    mapping = {}
    df0 = df.groupby(['WATER_SYSTEM_NO']).mean()
    mapping['Total Residential Population Served: '] = df0['Residential Population'].sum()
    mapping['Total Non Transient Population Served: '] = df0['Non Transient Population'].sum()
    mapping['Total Transient Population Served: '] = df0['Transient Population'].sum()
    mapping['Total Population Served: '] = df0['Total Population'].sum()
   # mapping['Average Number of Service Connections: '] = df0['Total Number of Service Connections'].mean().astype(int)
    return pd.DataFrame.from_dict(mapping, orient='index', columns= ["County Name : " + county_name])

In [23]:
# A helper function that converts panda table to beatiful html.
# Credit to: https://stackoverflow.com/questions/47704441/applying-styling-to-pandas-dataframe-saved-to-html-file
def to_pretty_html(df, title=''):
    '''
    Write an entire dataframe to an HTML file with nice formatting.
    '''

    result = '''
<html>
<head>
<style>

    h2 {
        text-align: center;
        font-family: Helvetica, Arial, sans-serif;
    }
    table { 
        margin-left: auto;
        margin-right: auto;
    }
    table, th, td {
        border: 1px solid black;
        border-collapse: collapse;
    }
    th, td {
        padding: 5px;
        text-align: center;
        font-family: Helvetica, Arial, sans-serif;
        font-size: 90%;
    }
    table tbody tr:hover {
        background-color: #dddddd;
    }
    .wide {
        width: 90%; 
    }

</style>
</head>
<body>
    '''
    result += '<h2> %s </h2>\n' % title
    result += df.to_html(classes='wide', escape=False, header = False)
    result += '''
</body>
</html>
'''
    return result

In [24]:
### Initialize the map

m = Map(center=(37.871593, -122.272743), zoom=5)
with open('json/ca_boundary.json', 'r') as f:
    data = json.load(f)
geo_json = GeoJSON(data=data)
click_marker = None
prev_marker_layer = None
m.add_layer(geo_json)
m.add_control(FullScreenControl())
check_widget = WidgetControl(widget=info_widgets, position='bottomleft')
m.add_control(check_widget)
m.add_control(WidgetControl(widget=click_widget, position='bottomleft'))
m.add_control(measure)
pd.set_option("display.max_rows", 10)
water_icon = Icon(icon_url='./images/water_icon.png', icon_size=[38, 38], icon_anchor=[22,94])


In [25]:
## Define the main logic for handling clicks

def handle_click(**kwargs):
    if kwargs.get('type') == 'click':
        if (click_widget.value == False):
            click_lat_long = kwargs.get('coordinates')
            global click_marker
            if click_marker != None:
                m.remove_layer(click_marker)
            click_marker = Marker(location= [click_lat_long[0], click_lat_long[1]], draggable =False)
            m.add_layer(click_marker)
            request_query = "https://geo.fcc.gov/api/census/area?lat=" + \
                str(click_lat_long[0]) + "&lon=" + str(click_lat_long[1]) +"&format=json"
            response_json = json.loads(requests.get(request_query).text) 
            if 'results' in response_json.keys():
                if response_json['results'] != []:
                    state_name = response_json['results'][0]['state_name']
                    county_name = response_json['results'][0]['county_name']
                    end = "<p> Sorry, but I have no data about states other than California </p>" if state_name != 'California' else ""
                    info_widgets.value = "<p>The latitude, longitude for your click: <b>" + str(tuple(click_lat_long)) + "</b><br>" + \
                    "This corresponds to the county: <b>" + county_name + "</b> in " + str(state_name) + "</p>"  + end

                    if state_name == 'California':
                        global prev_marker_layer
                        if prev_marker_layer != None:
                            m.remove_layer(prev_marker_layer)
                        county_name = county_name.upper()
                        want_columns = ['WATER_SYSTEM_NO', 'Water System Name','FACILITY_NAME','LATITUDE_MEASURE', 
                                        'LONGITUDE_MEASURE', 'Primary Water Source Type', 'REG_AGENCY',
                                       'Water System Status', 'Residential Population', 'Non Transient Population', 
                                       'Transient Population', 'Total Population', 'Total Number of Service Connections']
                        found_rows = primary_merged[primary_merged['Principal County Served'] == county_name][want_columns]
                        if len(found_rows) > 0:
                            summary = summarize(found_rows, county_name)
                            info_widgets.value += "<p> I found <b>"+ str(len(found_rows))+ "</b> water facilities that belong to <b>" + str(len(summary))+ "</b> water systems</p>"
                            info_widgets_value_cpy = info_widgets.value[:]
                            info_widgets.value += "<p> Loading these water facilities... ... </p>"
                            #start = time.time()
                            lat_long_well_info_matrix =  found_rows.values[: , 3:5]
                            info_arr = np.array(found_rows.apply(lambda x: to_pretty_html(x.to_frame()), axis = 1))
                            info_arr = info_arr.reshape(info_arr.shape[0], 1)
                            lat_long_well_info_matrix = np.append(lat_long_well_info_matrix, info_arr,axis = 1)
                            #all_markers = Parallel(n_jobs=2)(delayed(create_marker_from_row)(r) for r in lat_long_well_info_matrix)
                            #pool = mp.Pool(processes=4)
                            all_markers = [create_marker_from_row(r) for r in lat_long_well_info_matrix]
                            cluster_layer = MarkerCluster(markers = all_markers)
                            prev_marker_layer = cluster_layer
                            m.add_layer(cluster_layer)
                            info_widgets.value = info_widgets_value_cpy + "<p> Loading done! Try clicking on the markers for more info about each water facility </p>"
                            #end = time.time()
                            #print("this takes " + str(end - start) + " seconds!")
            
def create_marker_from_row(found_row):
    source_lat, source_long = found_row[0], found_row[1]
    well_info = HTML(layout=Layout(width="20px"))
    well_info.value = found_row[2]
    temp =  Marker(location= [float(source_lat), float(source_long)], popup = well_info, draggable =False, icon = water_icon)
    return temp
    
m.on_interaction(handle_click)

In [26]:
display(m)

TWFwKGJhc2VtYXA9eyd1cmwnOiAnaHR0cHM6Ly97c30udGlsZS5vcGVuc3RyZWV0bWFwLm9yZy97en0ve3h9L3t5fS5wbmcnLCAnbWF4X3pvb20nOiAxOSwgJ2F0dHJpYnV0aW9uJzogJ01hcCDigKY=
